# Giới thiệu

DrQA là một hệ thống cho việc đọc hiểu được áp dụng để trả lời câu hỏi mã nguồn mở. Cụ thể, DrQA nhằm mục đích giải quyết nhiệm vụ đọc máy quy mô lớn. Trong phần cài đặt, chúng tôi tìm kiếm câu trả lời của một câu hỏi trong một kho tài liệu có khả năng cao của các tài liệu phi cấu trúc. Vì thế, hệ thống phải kết hợp cả thách thức trích xuất và hiểu máy một đoạn văn bản.

Thực nghiệm với DrQA tập trung vào việc trả lời các câu hỏi thực tế trong khi sử dụng Wikipedia như nguồn kiến thức duy nhất cho các tài liệu. Wikipedia là một nguồn phong phú, chi tiết, phù hợp quy mô lớn. Để trả lời bất kỳ câu hỏi nào, đầu tiên phải trích xuất các bài báo chung trong hơn 5 triệu bài, và sau đó quét chúng cẩn thẩn, để xác định câu trả lời.

Lưu ý rằng DrQA coi Wikipeadia như tập hợp các bài báo chung và không dựa vào cấu trúc đồ thị bên trong. Do đó, **DrQA có thể được áp dụng dễ dàng cho bất kỳ bộ sưu tập tài liệu nào**. Trong phần trích xuất tài liệu README có mô tả chi tiết.

Kho tài liệu này bao gồm code, dữ liệu và mô hình tiền huấn luyện cho tiền xử lý và truy vấn Wikipedia như được mô tả trong bài báo -- Xem Mô hình được train và dữ liệu. Chúng tôi cũng liệt kê một số bộ dữ liệu khác nhau cho việc đánh giá, xem phần các bộ dữ liệu QA. Lưu ý rằng công việc này là được cấu trúc lại và hiệu quả hơn phiên bản code gốc. Các con số sản phẩm rất giống nhau nhưng không hoàn toàn.

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ProjectNLP/NLP_Perfect/DrQA')
print('Kết nối đến thư mục code thành công') 

Kết nối đến thư mục code thành công


## Cài đặt drqa

DrQA yêu cầu sử dụng Linux/OSX và python 3.5 trở lên. Nó cũng yêu cầu PyTorch phiên bản 1.0. Nó dựa trên các thư viện khác được liệt kê trong file requirements.txt. CUDA là được yêu cầu mạnh mẽ cho tốc độ, nhưng không bắt buộc.

Chạy câu lệnh sau đây để clone reposity về và cài đặt DrQA:

```
git clone https://github.com/facebookresearch/DrQA.git
cd DrQA; pip install -r requirements.txt; python setup.py develop
```

Lưu ý: requirements.txt bao gồm một tập con tất cả các gói có thể được yêu cầu. 

Tùy thuộc vào cái bạn muốn chạy, bạn có thể cần cài đặt thêm các gói( ví dụ spacy)

Nếu bạn sử dụng CoreNLP Tokenizer hoặc SpacyTokenizer bạn sẽ cần cài đặt Stanford CoreNLP jars và mô hình spaCy en tương ứng. Nếu bạn dùng Stanford CoreNLP, có jars trong biến môi trường CLASSPATH trong java của bạn, hoặc đặt đường dẫn tự động với:

```
import drqa.tokenizers
drqa.tokenizers.set_default('corenlp_classpath', 'your/corenlp/classpath/*')
```

**Quan trọng: Tokenizer mặc định là CoreNLP nên bạn sẽ cần điều đó trong CLASSPATH để chạy các ví dụ ở phần README**

Vd: ```export CLASSPATH=$CLASSPATH:/path/to/corenlp/download/* ```

Nếu bạn chưa cài đặt CoreNLP thì có thể chạy:
```
./install_corenlp.sh
```

Xác nhận xem nó có chạy không bằng :

```
from drqa.tokenizers import CoreNLPTokenizer
tok = CoreNLPTokenizer()
tok.tokenize('hello world').words() # Should complete immediately
```

Để thuận tiện, mô hình trình đọc tài liệu, trích xuất và Pipeline sẽ cố gắng load các mô hình mặc định nếu không có đối số mô hình được truyền vào. Xem phía dưới cho việc cài đặt các mô hình này.

In [ ]:
pip install elasticsearch

     |████████████████████████████████| 358kB 9.2MB/s 


In [ ]:
import numpy
import sklearn
import termcolor
import regex
import tqdm
import prettytable
import scipy
import nltk
import elasticsearch
import spacy
# import pexpect==4.2.1

In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
import drqa.tokenizers
drqa.tokenizers.set_default('corenlp_classpath', '/content/drive/MyDrive/Đồ án cuối kì - Xử lý ngôn ngữ tự nhiên/NLP- perfect version/DrQA/download_data/')
print('OK')

OK


In [ ]:
from drqa.tokenizers import spacy_tokenizer

In [ ]:
import sys
sys.path.append('/.../application/app/folder')
import file_1

import sys
sys.path.insert(1, '/.../application/app/folder')
import file

In [ ]:
%%shell
#!/bin/bash
# Copyright 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

set -e


# export CLASSPATH=$CLASSPATH:/content/drive/MyDrive/Đồ án cuối kì - Xử lý ngôn ngữ tự nhiên/NLP- perfect version/DrQA/download_data/*
# By default download to the data directory I guess
read -p "Specify download path or enter to use default (data/corenlp): " path
DOWNLOAD_PATH="${path:-data/corenlp}"
echo "Will download to: $DOWNLOAD_PATH"

# Download zip, unzip
pushd "/tmp"
wget -O "stanford-corenlp-full-2017-06-09.zip" "http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip"
unzip "stanford-corenlp-full-2017-06-09.zip"
rm "stanford-corenlp-full-2017-06-09.zip"
popd

# Put jars in DOWNLOAD_PATH
mkdir -p "$DOWNLOAD_PATH"
mv "/tmp/stanford-corenlp-full-2017-06-09/"*".jar" "$DOWNLOAD_PATH/"

# Append to bashrc, instructions
while read -p "Add to ~/.bashrc CLASSPATH (recommended)? [yes/no]: " choice; do
    case "$choice" in
        yes )
            echo "export CLASSPATH=\$CLASSPATH:$DOWNLOAD_PATH/*" >> ~/.bashrc;
            break ;;
        no )
            break ;;
        * ) echo "Please answer yes or no." ;;
    esac
done

printf "\n*** NOW RUN: ***\n\nexport CLASSPATH=\$CLASSPATH:$DOWNLOAD_PATH/*\n\n****************\n"


Specify download path or enter to use default (data/corenlp): /content/drive/MyDrive/Đồ án cuối kì - Xử lý ngôn ngữ tự nhiên/NLP- perfect version/DrQA/download_data/*
Will download to: /content/drive/MyDrive/Đồ án cuối kì - Xử lý ngôn ngữ tự nhiên/NLP- perfect version/DrQA/download_data/*
/tmp /content
--2021-06-21 07:44:44--  http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip [following]
--2021-06-21 07:44:44--  https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/softwa

In [ ]:
from drqa.tokenizers import SpacyTokenizer
tok = SpacyTokenizer()
tok.tokenize('hello world').words() # Should complete immediately

['hello', 'world']

In [ ]:
%%shell
#!/bin/bash
# Copyright 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

set -e

# Configure download location
DOWNLOAD_PATH="/content/drive/MyDrive/Đồ án cuối kì - Xử lý ngôn ngữ tự nhiên/NLP- perfect version/DrQA/download_data"

# Get AWS hosted data
DOWNLOAD_PATH_TAR="$DOWNLOAD_PATH.tar.gz"

# Download main hosted data
# wget -O "$DOWNLOAD_PATH_TAR" "https://dl.fbaipublicfiles.com/drqa/data.tar.gz"

# Untar
tar -xvf "$DOWNLOAD_PATH_TAR"

# Remove tar ball
rm "$DOWNLOAD_PATH_TAR"

# Get externally hosted data
DATASET_PATH="/content/drive/MyDrive/Đồ án cuối kì - Xử lý ngôn ngữ tự nhiên/NLP- perfect version/DrQA/download_data/datasets"

# Get SQuAD train
wget -O "$DATASET_PATH/SQuAD-v1.1-train.json" "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
python scripts/convert/squad.py "$DATASET_PATH/SQuAD-v1.1-train.json" "$DATASET_PATH/SQuAD-v1.1-train.txt"

# Get SQuAD dev
wget -O "$DATASET_PATH/SQuAD-v1.1-dev.json" "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
python scripts/convert/squad.py "$DATASET_PATH/SQuAD-v1.1-dev.json" "$DATASET_PATH/SQuAD-v1.1-dev.txt"

# Download official eval for SQuAD
curl "https://worksheets.codalab.org/rest/bundles/0xbcd57bee090b421c982906709c8c27e1/contents/blob/" >  "./scripts/reader/official_eval.py"

# Get WebQuestions train
wget -O "$DATASET_PATH/WebQuestions-train.json.bz2" "http://nlp.stanford.edu/static/software/sempre/release-emnlp2013/lib/data/webquestions/dataset_11/webquestions.examples.train.json.bz2"
bunzip2 -f "$DATASET_PATH/WebQuestions-train.json.bz2"
python scripts/convert/webquestions.py "$DATASET_PATH/WebQuestions-train.json" "$DATASET_PATH/WebQuestions-train.txt"
rm "$DATASET_PATH/WebQuestions-train.json"

# Get WebQuestions test
wget -O "$DATASET_PATH/WebQuestions-test.json.bz2" "http://nlp.stanford.edu/static/software/sempre/release-emnlp2013/lib/data/webquestions/dataset_11/webquestions.examples.test.json.bz2"
bunzip2 -f "$DATASET_PATH/WebQuestions-test.json.bz2"
python scripts/convert/webquestions.py "$DATASET_PATH/WebQuestions-test.json" "$DATASET_PATH/WebQuestions-test.txt"
rm "$DATASET_PATH/WebQuestions-test.json"

# Get freebase entities for WebQuestions
wget -O "$DATASET_PATH/freebase-entities.txt.gz" "https://dl.fbaipublicfiles.com/drqa/freebase-entities.txt.gz"
gzip -d "$DATASET_PATH/freebase-entities.txt.gz"

echo "DrQA download done!"


## Đào tạo mô hình và dữ liệu

Để tải tất cả các mô hình đã đào tạo được cung cấp và dữ liệu cho trả lời câu hỏi Wikipedia, chạy:

```
./download.sh
```
*Cảnh báo: this downloads a 7.5GB tarball (25GB untarred) và sẽ mất một khoảng thời gian.*

Cấu trúc, vị trí tải:



```
DrQA
├── data (or $DRQA_DATA)
    ├── datasets
    │   ├── SQuAD-v1.1-<train/dev>.<txt/json>
    │   ├── WebQuestions-<train/test>.txt
    │   ├── freebase-entities.txt
    │   ├── CuratedTrec-<train/test>.txt
    │   └── WikiMovies-<train/test/entities>.txt
    ├── reader
    │   ├── multitask.mdl
    │   └── single.mdl
    └── wikipedia
        ├── docs.db
        └── docs-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz
```

Các đường dẫn mô hình mặc định cho các mô đun khác nhau cũng có thể được chỉnh sửa tự động trong code, ví dụ:

```
import drqa.reader
drqa.reader.set_default('model', '/path/to/model')
reader = drqa.reader.Predictor() # Mô hình mặc định được tải cho việc dự đoán
```


In [ ]:
import drqa.reader
drqa.reader.set_default('model', '/content/drive/MyDrive/ProjectNLP/NLP_Perfect/DrQA/data/reader/single.mdl')
reader = drqa.reader.Predictor() # Mô hình mặc định được tải cho việc dự đoán
print('chạy xong')

## Trình trích xuất tài liệu

Mô hình TF-IDF sử dụng Wikipedia( unigrams và bigrams, 2^24 bins, tokenization đơn giản), được đánh giá trên nhiều bộ dữ liệu( bộ test, bộ dev cho SQuAD):

| Model | SQuAD P@5 | CuratedTREC P@5 | WebQuestions P@5 | WikiMovies P@5 | Size |
| :---: | :-------: | :-------------: | :--------------: | :------------: | :---: |
| [TF-IDF model](https://dl.fbaipublicfiles.com/drqa/docs-tfidf-ngram%3D2-hash%3D16777216-tokenizer%3Dsimple.npz.gz) | 78.0 | 87.6 | 75.0 | 69.8 | ~13GB |

P@5 ở đây là được định nghĩa như % các câu hỏi cho các phần câu trả lời xuất hiện trong top5 tài liệu

## Trình đọc tài liệu

Mô hinh được huấn luyện chỉ trên bộ SQuAD, được đánh giá trong cài đặt SQuAD:

| Model | SQuAD Dev EM | SQuAD Dev F1 | Size |
| :---: | :-----------:| :----------: | :--: |
| [Single model](https://dl.fbaipublicfiles.com/drqa/single.mdl) | 69.4 | 78.9 | ~130MB |

Mô hình được đào tạo với giám sát từ xa mà không có các tính năng NER/POS/lemma, được đánh giá trên nhiều bộ dữ liệu( bộ test, bộ dev cho SQuAD) trong cài đặt đầy đủ Wikipedia:

| Model | SQuAD EM | CuratedTREC EM | WebQuestions EM | WikiMovies EM | Size |
| :---: | :------: | :------------: | :-------------: | :-----------: | :--:
| [Multitask model](https://dl.fbaipublicfiles.com/drqa/multitask.mdl) | 29.5 | 27.2 | 18.5 | 36.9 | ~270MB |



## Wikipedia

Các thử nghiệm quy mô đầy đủ của chúng tôi được thực hiện trên Wikipedia tiếng Anh 2016-12-21. Kết xuất được xử lý bằng WikiExtractor và được lọc cho các trang nội bộ, danh sách, chỉ mục và phác thảo (các trang thường chỉ là liên kết). Chúng tôi lưu trữ các tài liệu trong cơ sở dữ liệu sqlite mà drqa.retriever.DocDB cung cấp một giao diện.

| Database | Num. Documents | Size |
| :------: | :------------: | :-----------------: |
| [Wikipedia](https://dl.fbaipublicfiles.com/drqa/docs.db.gz) | 5,075,182 | ~13GB |

## QA Datasets

Các bộ dữ liệu được sử dụng cho đào tạo và đánh giá DrQA có thể được tìm thấy ở đây:

- SQuAD: [train](https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json), [dev](https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json)
- WebQuestions: [train](http://nlp.stanford.edu/static/software/sempre/release-emnlp2013/lib/data/webquestions/dataset_11/webquestions.examples.train.json.bz2), [test](http://nlp.stanford.edu/static/software/sempre/release-emnlp2013/lib/data/webquestions/dataset_11/webquestions.examples.test.json.bz2), [entities](https://dl.fbaipublicfiles.com/drqa/freebase-entities.txt.gz)
- WikiMovies: [train/test/entities](https://dl.fbaipublicfiles.com/drqa/WikiMovies.tar.gz)
(Rehosted in expected format from https://research.fb.com/downloads/babi/)
- CuratedTrec: [train/test](https://dl.fbaipublicfiles.com/drqa/CuratedTrec.tar.gz)
(Rehosted in expected format from https://github.com/brmson/dataset-factoid-curated)

**Định dạng A**

Các tập lệnh
    retriever/eval.py, pipeline/eval.py
và 
    distant/generate.py

mong đợi các bộ dữ liệu như file .txt mà mỗi dòng là một cặp QA được mã hóa JSON, giống như này:

```python
'{"question": "q1", "answer": ["a11", ..., "a1i"]}'
...
'{"question": "qN", "answer": ["aN1", ..., "aNi"]}'
```

Các tập lệnh để chuyển SQuAD và WebQuestion thành định dạng này là được bao gồm trong
    scripts/convert
Điều này tự động làm trong *download.sh*

**Định dạng B**

Các tập lệnh thư mục *reader* mong đợi các tập dữ liệu dưới dạng tệp .json, nơi dữ liệu được sắp xếp giống SQuAD:

```
file.json
├── "data"
│   └── [i]
│       ├── "paragraphs"
│       │   └── [j]
│       │       ├── "context": "paragraph text"
│       │       └── "qas"
│       │           └── [k]
│       │               ├── "answers"
│       │               │   └── [l]
│       │               │       ├── "answer_start": N
│       │               │       └── "text": "answer"
│       │               ├── "id": "<uuid>"
│       │               └── "question": "paragraph question?"
│       └── "title": "document id"
└── "version": 1.1
```

## Danh sách thực thể
Một số bộ dữ liệu có danh sách ứng viên (có thể lớn) để chọn câu trả lời. Ví dụ: câu trả lời của WikiMovies là các mục nhập OMDb trong khi WebQuestions dựa trên Freebase. Nếu chúng tôi biết các ứng cử viên, chúng tôi có thể áp đặt rằng tất cả các câu trả lời được dự đoán phải nằm trong danh sách này bằng cách loại bỏ bất kỳ khoảng(span) nào mà điểm cao hơn là không có.

In [ ]:
import sys
sys.path.append('/.../application/app/folder')
!python scripts/reader/interactive.py --model 

In [ ]:
i = []
while True:
    i.append(i)

In [ ]:
! python /content/drive/MyDrive/ProjectNLP/NLP_Perfect/DrQA/scripts/retriever/interactive.py

06/25/2021 02:51:43 AM: [ Initializing ranker... ]
06/25/2021 02:51:43 AM: [ Loading /content/drive/MyDrive/ProjectNLP/NLP_Perfect/DrQA/data/wikipedia/docs-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz ]
tcmalloc: large alloc 8506228736 bytes == 0x563af902a000 @  0x7fefb7cf91e7 0x7fefb3b9d46e 0x7fefb3bedc7b 0x7fefb3ba0ce8 0x563af2ddfc65 0x563af2da0559 0x563af2e144f8 0x563af2e0e4ae 0x563af2da13ea 0x563af2e1032a 0x563af2da33cb 0x563af2de4599 0x563af2de450c 0x563af2e874d9 0x563af2e0f91e 0x563af2da130a 0x563af2e137f0 0x563af2e0e4ae 0x563af2da1c9f 0x563af2de2d79 0x563af2ddfcc4 0x563af2da0559 0x563af2e144f8 0x563af2e0e4ae 0x563af2e0e1b3 0x563af2ed8182 0x563af2ed84fd 0x563af2ed83a6 0x563af2eaf723 0x563af2eaf3cc 0x7fefb6ae3bf7
tcmalloc: large alloc 4253114368 bytes == 0x563cf491a000 @  0x7fefb7cf91e7 0x7fefb3b9d46e 0x7fefb3bedc7b 0x7fefb3ba0ce8 0x563af2ddfc65 0x563af2da0559 0x563af2e144f8 0x563af2e0e4ae 0x563af2da13ea 0x563af2e1032a 0x563af2da33cb 0x563af2de4599 0x563af2de450c 0x563af2e874d

In [ ]:
! python scripts/retriever/eval.py /path/to/format/A/dataset.txt --model /path/to/model

# DrQA Components

## Document Retriever

DrQA không bị ràng buộc bởi bất kỳ loại hệ thống truy xuất cụ thể nào -- miễn là nó thu hẹp không gian tìm kiếm một cách hiệu quả và tập trung vào các tài liệu liên quan.

Theo các hệ thống QA cổ điển, chúng tôi bao gồm một hệ thống truy xuất tài liệu dựa trên các vector tú từ có trọng số TF-IDF thưa thớt( không dùng máy học). Chúng tôi dùng thêm các túi được băm n-grams( ở đây là unigrams và bigrams).

Để xem cách xây dựng mô hình như vậy của riêng bạn trên các tài liệu mới, hãy xem README của trình trích xuất.

Để truy vấn Wikipedia có thể tương tác:

```
python scripts/retriever/interactive.py --model /path/to/model
```

Nếu *model* đã "rời khỏi", mô hình mặc định của chúng tôi sẽ được sử dụng( giả sử nó đã được tải).

Để đánh giá tỉ lệ trích xuất( % phù hợp trong top 5) trên bộ dữ liệu:

```
python scripts/retriever/eval.py /path/to/format/A/dataset.txt --model /path/to/model
```

## Document Reader

Trình đọc tài liệu của DrQA là một mô hình hiểu máy mạng thần kinh tái phát nhiều lớp được đào tạo để trả lời câu hỏi khai thác. Đó là, mô hình cố gắng tìm câu trả lời cho bất kỳ câu hỏi nào dưới dạng khoảng văn bản trong một trong các tài liệu được trả về.


Trình đọc Tài liệu được lấy cảm hứng từ và được đào tạo chủ yếu về tập dữ liệu SQuAD. Nó cũng có thể được sử dụng độc lập trên các tác vụ giống SQuAD như vậy, trong đó ngữ cảnh cụ thể được cung cấp cùng với câu hỏi, câu trả lời được chứa trong ngữ cảnh.


Để xem cách đào tạo Trình đọc tài liệu trên SQuAD, hãy xem trình đọc README.


Để tương tác đặt câu hỏi về văn bản với một mô hình được đào tạo: 

```
python scripts/reader/interactive.py --model /path/to/model
```

Một lần nữa, *model* ở đây là tùy chọn; một **model mặc định(có link)** được dùng nếu nó không được chỉ định.

Để chạy mô hình dự đoán trên một bộ dữ liệu:

```
python scripts/reader/predict.py /path/to/format/B/dataset.json --model /path/to/model
```

## DrQA Pipeline

Toàn bộ hệ thống được liên kết cùng nhau trong
    drqa.pipeline.DrQA
Để đặt câu hỏi tương tác sử dụng DrQA đầy đủ:

```
python scripts/pipeline/interactive.py
```

Các đối số tùy chọn:

```
--reader-model    Path to trained Document Reader model.
--retriever-model Path to Document Retriever model (tfidf).
--doc-db          Path to Document DB.
--tokenizer      String option specifying tokenizer type to use (e.g. 'corenlp').
--candidate-file  List of candidates to restrict predictions to, one candidate per line.
--no-cuda         Use CPU only.
--gpu             Specify GPU device id to use.
```

Để chạy dự đoán trên bộ dữ liệu:

```
python scripts/pipeline/predict.py /path/to/format/A/dataset.txt
```

Các đối số tùy chọn:

```
--out-dir             Directory to write prediction file to (<dataset>-<model>-pipeline.preds).
--reader-model        Path to trained Document Reader model.
--retriever-model     Path to Document Retriever model (tfidf).
--doc-db              Path to Document DB.
--embedding-file      Expand dictionary to use all pretrained embeddings in this file (e.g. all glove vectors to minimize UNKs at test time).
--candidate-file      List of candidates to restrict predictions to, one candidate per line.
--n-docs              Number of docs to retrieve per query.
--top-n               Number of predictions to make per query.
--tokenizer           String option specifying tokenizer type to use (e.g. 'corenlp').
--no-cuda             Use CPU only.
--gpu                 Specify GPU device id to use.
--parallel            Use data parallel (split across GPU devices).
--num-workers         Number of CPU processes (for tokenizing, etc).
--batch-size          Document paragraph batching size (Reduce in case of GPU OOM).
--predict-batch-size  Question batching size (Reduce in case of CPU OOM).
```

## Giám sát từ xa (DS)

Hiệu suất của DrQA cải thiện đáng kể trong chế độ cài đặt đầy đủ khi được cung cấp dữ liệu giám sát từ xa từ các bộ dữ liệu bổ sung. Đưa ra các cặp câu hỏi-câu trả lời nhưng không có ngữ cảnh hỗ trợ, chúng ta có thể sử dụng phương pháp heuristics đối sánh chuỗi để tự động liên kết các đoạn văn với các ví dụ đào tạo này.

>Question: What U.S. state’s motto is “Live free or Die”?
>
>Answer: New Hampshire
>
>DS Document: Live Free or Die

Các tập lệnh
    scripts/distant
chứa mã để tạo và kiểm tra dữ liệu được giám sát từ xa như vậy. Chi tiết hơn có thể xem DS README.

## Tokenizers

Chúng tôi cung cấp một số tùy chọn tokenizer khác nhau cho sự thuận tiện. Mỗi cái có ưu nhược điểm dựa trên bao nhiêu thư viện được yêu cầu, chi phí để chạy, tốc độ và hiệu suất. Đối với các thử nghiệm được báo cáo của chúng tôi, chúng tôi đã sử dụng CoreNLP (nhưng tất cả các kết quả đều tương tự). 

Xem các danh sách tùy chọn trong class tokenizers

## Có thể tạo giao diện để dùng thông qua Github được viết sẵn.